# Optimal metrics on the space of rankings for Social Choice Theory

### Francesco Colasanto & Davide Fabbrico

The analysis of properties of Social Function is focal point of Social Choice Theory. In the Arrow's framework there is a group of individuals who has to choose among a set $\{1,..., k\} =: A_k$ of alternatives. Each individual gives his vote through a linear order on $A_k$. A Social Function takes as input an electoral profile and returns the winner alternatives. There is a large class of Social Functions that can be defined starting from a "metric" on the space of rankings $L(A_k)$ with alternatives $A_k$, for instance the Kemeny's Social Preference Function, the Score Social Choice Functions as the Plurality or the Borda's one. 

In [35]:
import numpy as np
import math
import itertools
from functools import lru_cache
from sympy.combinatorics import Permutation
from tqdm import tqdm

class SimplexPSO:
    def __init__(self, n_particles, dim, objective_func, objective_params, max_iter=100):
        self.n_particles = n_particles
        self.dim = dim
        self.objective_func = objective_func
        self.objective_params = objective_params
        self.max_iter = max_iter

        # Inizializza parametri PSO
        self.omega = 0.9
        self.c1 = 2.0
        self.c2 = 2.0
        
        # Inizializza posizioni e velocità
        self.positions = np.random.dirichlet(np.ones(dim), size=n_particles)
        self.velocities = np.zeros((n_particles, dim))
        
        self.initialize_best_positions()

    def initialize_best_positions(self):
        self.pbest = self.positions.copy()
        self.pbest_scores = np.array([self.objective_func(p, **self.objective_params)
                                    for p in self.positions])
        best_idx = np.argmin(self.pbest_scores)
        self.gbest = self.pbest[best_idx]
        self.gbest_score = self.pbest_scores[best_idx]

    def project_simplex(self, x):
        x_sorted = np.sort(x)[::-1]
        cumsum = np.cumsum(x_sorted) - 1.0
        indices = np.arange(1, len(x) + 1)
        rho = np.where(x_sorted - cumsum / indices > 0)[0][-1]
        lambda_ = cumsum[rho] / (rho + 1)
        return np.maximum(x - lambda_, 0)

    def optimize(self):
        convergence = []
        for _ in tqdm(range(self.max_iter)):
            for i in range(self.n_particles):
                self.update_particle(i)
            self.update_inertia()
            convergence.append(self.gbest_score)
        return self.gbest, self.gbest_score, convergence

    def update_particle(self, idx):
        r1, r2 = np.random.rand(2)
        self.velocities[idx] = (self.omega * self.velocities[idx] +
                               self.c1 * r1 * (self.pbest[idx] - self.positions[idx]) +
                               self.c2 * r2 * (self.gbest - self.positions[idx]))
        
        new_pos = self.project_simplex(self.positions[idx] + self.velocities[idx])
        score = self.objective_func(new_pos, **self.objective_params)
        
        if score < self.pbest_scores[idx]:
            self.update_best_positions(idx, new_pos, score)

    def update_best_positions(self, idx, new_pos, score):
        self.pbest[idx] = new_pos
        self.pbest_scores[idx] = score
        if score < self.gbest_score:
            self.gbest = new_pos
            self.gbest_score = score

    def update_inertia(self):
        self.omega *= 0.99

def sparse_preference(vector, processor, n=500, MCiterations=100, eps=0.05):
    k = processor.k
    fact = math.factorial(k)
    total = 0

    for it in range(MCiterations):
        samples = np.random.choice(fact, n)
        min_vals = []

        chunk_size = 1000
        for chunk_start in range(0, fact, chunk_size):
            chunk = range(chunk_start, min(chunk_start+chunk_size, fact))
            sums = np.zeros(len(chunk))

            for i, idx in enumerate(chunk):
                row_sum = 0
                for j in samples:
                    if idx == 0 or j == 0:
                        val = vector[abs(idx-j)-1] if (idx !=0 or j!=0) else 0
                    else:
                        product = processor.perm_product(idx, j)
                        val = vector[product-1] if product !=0 else 0
                    row_sum += val
                sums[i] = row_sum

            min_vals.append(sums.min())

        current_min = np.min(min_vals)
        eps_value = eps * abs(current_min)
        # print(f"[Preference] Iter {it+1}/{MCiterations} — eps = {eps_value:.6f}")
        total += np.sum((sums >= current_min - eps_value) & (sums <= current_min + eps_value))

    return total / (MCiterations * len(min_vals))

def dynamic_choice(vector, processor, n=500, MCiterations=100, eps=0.05):
    k = processor.k
    vectorTmp = np.zeros(k)
    vectorTmp[1:] = vector  # stesso schema di Choice
    
    counts = []

    for it in range(MCiterations):
        sums = np.zeros(k)
        for _ in range(n):
            perm = np.random.permutation(k)
            for y in range(k):
                sums[y] += vectorTmp[perm[y]]
        
        min_sum = sums.min()
        eps_value = eps * abs(min_sum)
        
        # print(f"[Choice] Iter {it+1}/{MCiterations} — eps = {eps_value:.6f}")
        counts.append(np.sum((sums >= min_sum - eps_value) & (sums <= min_sum + eps_value)))

    return np.mean(counts)


class EfficientPermutationProcessor:
    def __init__(self, k):
        self.k = k
        self.rank_dict = {}
        self.perm_list = []
        
        for idx, perm in enumerate(itertools.permutations(range(k))):
            self.perm_list.append(perm)
            self.rank_dict[perm] = idx
    
    @property
    def fact(self):
        return math.factorial(self.k)
    
    @lru_cache(maxsize=None)
    def perm_product(self, i, j):
        """Calcola il prodotto delle permutazioni usando SymPy"""
        p1 = Permutation(list(self.perm_list[i])) 
        p2 = Permutation(list(self.perm_list[j]))
        product = p1 * p2
        return self.rank_dict[tuple(product.array_form)] 

In [36]:
if __name__ == "__main__":
    k = 3  
    n = 10
    
    # # Configurazione per Preference
    # pso_pref = SimplexPSO(
    #     n_particles=100,
    #     dim=math.factorial(k) - 1,
    #     objective_func=sparse_preference,
    #     objective_params={'processor': EfficientPermutationProcessor(k), 'n': n},
    #     max_iter=50
    # )
    
    # Configurazione per Choice
    pso_choice = SimplexPSO(
        n_particles=100,
        dim=k-1, 
        objective_func=dynamic_choice,
        objective_params={'processor': EfficientPermutationProcessor(k), 'n': n},
        max_iter=50
    )
    
    # Esegui ottimizzazioni
    # best_pref, score_pref, _ = pso_pref.optimize()
    best_choice, score_choice, _ = pso_choice.optimize()
    
    # print(f"Preference Score: {score_pref:.4f}")
    print(f"Choice Score: {score_choice:.4f}")

100%|███████████████████████████████████████████| 50/50 [00:10<00:00,  4.95it/s]

Choice Score: 1.0100


In [37]:
import numpy as np
import pandas as pd
import time
import math
import itertools

# Parametri simulazione
ks = [3, 5, 8]
ns = [10, 100, 1000]
epsilons = [0.1, 0.3, 0.5]
n_particles = 50
max_iter = 50
n_repeats = 5

# Inizializza raccolta risultati
all_results = []

# Loop su tutte le combinazioni (k, n, eps) con ripetizioni
for k, n, eps in itertools.product(ks, ns, epsilons):
    print(f"\n--- Simulazione per k={k}, n={n}, eps={eps} ---")
    
    for repeat in range(1, n_repeats + 1):
        print(f" Ripetizione {repeat}")
        processor = EfficientPermutationProcessor(k)

        # Define wrapped objective functions
        objective_pref = lambda v: sparse_preference(v, processor=processor, n=n, eps=eps)
        objective_choice = lambda v: dynamic_choice(v, processor=processor, n=n, eps=eps)

        # Dimensionalità
        dim_pref = math.factorial(k) - 1
        dim_choice = k - 1

        # Ottimizzazione preference
        start = time.time()
        pso_pref = SimplexPSO(n_particles, dim_pref, objective_pref, {}, max_iter)
        best_p, score_p, _ = pso_pref.optimize()
        time_pref = time.time() - start

        # Ottimizzazione choice
        start = time.time()
        pso_choice = SimplexPSO(n_particles, dim_choice, objective_choice, {}, max_iter)
        best_c, score_c, _ = pso_choice.optimize()
        time_choice = time.time() - start

        # Aggiungi risultato
        all_results.append({
            'k': k,
            'n': n,
            'eps': eps,
            'repeat': repeat,
            'score_pref': score_p,
            'score_choice': score_c,
            'time_pref': time_pref,
            'time_choice': time_choice
        })

# Salvataggio in CSV
df = pd.DataFrame(all_results)
df.to_csv("risultati_simulazione_pso.csv", index=False)
print("Simulazione completata e salvata in 'risultati_simulazione_pso.csv'")


--- Simulazione per k=3, n=10, eps=0.1 ---
 Ripetizione 1


100%|███████████████████████████████████████████| 50/50 [00:05<00:00,  9.69it/s]


 Ripetizione 2


100%|███████████████████████████████████████████| 50/50 [00:05<00:00,  9.41it/s]


 Ripetizione 3


...

 Ripetizione 3


100%|███████████████████████████████████████████| 50/50 [00:37<00:00,  1.33it/s]


 Ripetizione 4


 28%|████████████                               | 14/50 [00:10<00:26,  1.35it/s]


KeyboardInterrupt: 